### Workbook by Misha Salkinder
### Home Credit Default Risk Kaggle Competition
### Data Wrangling - Final Submission

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

In [3]:
def numeric_conversion(df, grouping_var, name):
    """
    Converts numerical columns to aggregated statistics by a grouping variable

    This function can be used by each of the supplemental financial history files that require grouping.
    """
        # Remove id variables other than grouping variable
    for col in df:
        if col != grouping_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[grouping_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[grouping_var] = group_ids
    
    agg = numeric_df.groupby(grouping_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
    
    # Need to create new column names
    columns = [grouping_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != grouping_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (name, var, stat))

    agg.columns = columns
    return agg

In [4]:
def categorical_conversion(df, grouping_var, name):
    """
    Groups categorical columns to columns of counts and of normalized counts for each feature.

    This function can be used by each of the supplemental financial history files that require grouping.
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[grouping_var] = df[grouping_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(grouping_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [5]:
# Reading the Credit Bureau file from csv
bureau = pd.read_csv('bureau.csv', index_col=None, engine='python')

In [6]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [7]:
bureau_num_agg = numeric_conversion(bureau.drop(columns = ['SK_ID_BUREAU']), 'SK_ID_CURR', 'bureau')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
bureau_num_agg.head()

,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_max,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_sum,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_max,bureau_CREDIT_DAY_OVERDUE_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_max,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_sum,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_max,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_sum
0,100001,7,-735.000000,-49,-1572,-5145,7,0.0,0,0,...,7,-93.142857,-6,-155,-652,7,3545.357143,10822.5,0.0,24817.5
1,100002,8,-874.000000,-103,-1437,-6992,8,0.0,0,0,...,8,-499.875000,-7,-1185,-3999,7,0.000000,0.0,0.0,0.0
2,100003,4,-1400.750000,-606,-2586,-5603,4,0.0,0,0,...,4,-816.000000,-43,-2131,-3264,0,NaN,NaN,NaN,0.0
3,100004,2,-867.000000,-408,-1326,-1734,2,0.0,0,0,...,2,-532.000000,-382,-682,-1064,0,NaN,NaN,NaN,0.0
4,100005,3,-190.666667,-62,-373,-572,3,0.0,0,0,...,3,-54.333333,-11,-121,-163,3,1420.500000,4261.5,0.0,4261.5


In [9]:
bureau_cat_agg = categorical_conversion(bureau.drop(columns = ['SK_ID_BUREAU']), 'SK_ID_CURR', 'bureau')

In [10]:
bureau_cat_agg.head()

,bureau_CREDIT_ACTIVE_Active_count,bureau_CREDIT_ACTIVE_Active_count_norm,bureau_CREDIT_ACTIVE_Bad debt_count,bureau_CREDIT_ACTIVE_Bad debt_count_norm,bureau_CREDIT_ACTIVE_Closed_count,bureau_CREDIT_ACTIVE_Closed_count_norm,bureau_CREDIT_ACTIVE_Sold_count,bureau_CREDIT_ACTIVE_Sold_count_norm,bureau_CREDIT_CURRENCY_currency 1_count,bureau_CREDIT_CURRENCY_currency 1_count_norm,...,bureau_CREDIT_TYPE_Microloan_count,bureau_CREDIT_TYPE_Microloan_count_norm,bureau_CREDIT_TYPE_Mobile operator loan_count,bureau_CREDIT_TYPE_Mobile operator loan_count_norm,bureau_CREDIT_TYPE_Mortgage_count,bureau_CREDIT_TYPE_Mortgage_count_norm,bureau_CREDIT_TYPE_Real estate loan_count,bureau_CREDIT_TYPE_Real estate loan_count_norm,bureau_CREDIT_TYPE_Unknown type of loan_count,bureau_CREDIT_TYPE_Unknown type of loan_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [11]:
# Reading the Bureau Balance file from csv
bureau_balance = pd.read_csv('bureau_balance.csv', index_col=None, engine='python')
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [12]:
bureau_balance_num_agg = numeric_conversion(bureau_balance, 'SK_ID_BUREAU', 'bureau_balance')
bureau_balance_num_agg.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_BUREAU,bureau_balance_MONTHS_BALANCE_count,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_sum
0,5001709,97,-48.0,0,-96,-4656
1,5001710,83,-41.0,0,-82,-3403
2,5001711,4,-1.5,0,-3,-6
3,5001712,19,-9.0,0,-18,-171
4,5001713,22,-10.5,0,-21,-231


In [13]:
bureau_balance_cat_agg = categorical_conversion(bureau_balance, 'SK_ID_BUREAU', 'bureau_balance')
bureau_balance_cat_agg.head()

,bureau_balance_STATUS_0_count,bureau_balance_STATUS_0_count_norm,bureau_balance_STATUS_1_count,bureau_balance_STATUS_1_count_norm,bureau_balance_STATUS_2_count,bureau_balance_STATUS_2_count_norm,bureau_balance_STATUS_3_count,bureau_balance_STATUS_3_count_norm,bureau_balance_STATUS_4_count,bureau_balance_STATUS_4_count_norm,bureau_balance_STATUS_5_count,bureau_balance_STATUS_5_count_norm,bureau_balance_STATUS_C_count,bureau_balance_STATUS_C_count_norm,bureau_balance_STATUS_X_count,bureau_balance_STATUS_X_count_norm
SK_ID_BUREAU,,,,,,,,,,,,,,,,
5001709,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402
5001710,5,0.060241,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446
5001711,3,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000
5001712,10,0.526316,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000
5001713,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000


In [14]:
# Dataframe grouped by the loan
bureau_balance_all = bureau_balance_num_agg.merge(bureau_balance_cat_agg, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')

# Merge to include the SK_ID_CURR
bureau_balance_all = bureau[['SK_ID_BUREAU', 'SK_ID_CURR']].merge(bureau_balance_all, on = 'SK_ID_BUREAU', how = 'left')

# Aggregate the stats for each client
bureau_balance_by_client = numeric_conversion(bureau_balance_all.drop(columns = ['SK_ID_BUREAU']), 'SK_ID_CURR','bb')

In [15]:
bureau_balance_by_client.head()

,SK_ID_CURR,bb_bureau_balance_MONTHS_BALANCE_count_count,bb_bureau_balance_MONTHS_BALANCE_count_mean,bb_bureau_balance_MONTHS_BALANCE_count_max,bb_bureau_balance_MONTHS_BALANCE_count_min,bb_bureau_balance_MONTHS_BALANCE_count_sum,bb_bureau_balance_MONTHS_BALANCE_mean_count,bb_bureau_balance_MONTHS_BALANCE_mean_mean,bb_bureau_balance_MONTHS_BALANCE_mean_max,bb_bureau_balance_MONTHS_BALANCE_mean_min,...,bb_bureau_balance_STATUS_X_count_count,bb_bureau_balance_STATUS_X_count_mean,bb_bureau_balance_STATUS_X_count_max,bb_bureau_balance_STATUS_X_count_min,bb_bureau_balance_STATUS_X_count_sum,bb_bureau_balance_STATUS_X_count_norm_count,bb_bureau_balance_STATUS_X_count_norm_mean,bb_bureau_balance_STATUS_X_count_norm_max,bb_bureau_balance_STATUS_X_count_norm_min,bb_bureau_balance_STATUS_X_count_norm_sum
0,100001,7,24.571429,52.0,2.0,172.0,7,-11.785714,-0.5,-25.5,...,7,4.285714,9.0,0.0,30.0,7,0.214590,0.500000,0.0,1.502129
1,100002,8,13.750000,22.0,4.0,110.0,8,-21.875000,-1.5,-39.5,...,8,1.875000,3.0,0.0,15.0,8,0.161932,0.500000,0.0,1.295455
2,100003,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.000000
3,100004,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.000000
4,100005,3,7.000000,13.0,3.0,21.0,3,-3.000000,-1.0,-6.0,...,3,0.666667,1.0,0.0,2.0,3,0.136752,0.333333,0.0,0.410256


In [16]:
bureau_balance_by_client.shape

(305811, 106)

In [17]:
import gc

del bureau_balance_all, bureau_balance_cat_agg, bureau_balance_num_agg, bureau_balance, bureau     # memory release
gc.collect()

91

In [18]:
# Reading the Credit Card Balance file from csv
cc_balance = pd.read_csv('credit_card_balance.csv', index_col=None, engine='python')

In [19]:
cc_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [20]:
cc_balance_num_agg = numeric_conversion(cc_balance, 'SK_ID_PREV', 'cc_balance')
cc_balance_num_agg.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_PREV,cc_balance_MONTHS_BALANCE_count,cc_balance_MONTHS_BALANCE_mean,cc_balance_MONTHS_BALANCE_max,cc_balance_MONTHS_BALANCE_min,cc_balance_MONTHS_BALANCE_sum,cc_balance_AMT_BALANCE_count,cc_balance_AMT_BALANCE_mean,cc_balance_AMT_BALANCE_max,cc_balance_AMT_BALANCE_min,...,cc_balance_SK_DPD_count,cc_balance_SK_DPD_mean,cc_balance_SK_DPD_max,cc_balance_SK_DPD_min,cc_balance_SK_DPD_sum,cc_balance_SK_DPD_DEF_count,cc_balance_SK_DPD_DEF_mean,cc_balance_SK_DPD_DEF_max,cc_balance_SK_DPD_DEF_min,cc_balance_SK_DPD_DEF_sum
0,1000018,5,-4.0,-2,-6,-20,5,74946.285000,136695.420,38879.145,...,5,0.0,0,0,0,5,0.0,0,0,0
1,1000030,8,-4.5,-1,-8,-36,8,55991.064375,103027.275,0.000,...,8,0.0,0,0,0,8,0.0,0,0,0
2,1000031,16,-8.5,-1,-16,-136,16,52394.439375,154945.935,0.000,...,16,0.0,0,0,0,16,0.0,0,0,0
3,1000035,5,-4.0,-2,-6,-20,5,0.000000,0.000,0.000,...,5,0.0,0,0,0,5,0.0,0,0,0
4,1000077,11,-7.0,-2,-12,-77,11,0.000000,0.000,0.000,...,11,0.0,0,0,0,11,0.0,0,0,0


In [21]:
cc_balance_cat_agg = categorical_conversion(cc_balance, 'SK_ID_PREV', 'cc_balance')
cc_balance_cat_agg.head()

,cc_balance_NAME_CONTRACT_STATUS_Active_count,cc_balance_NAME_CONTRACT_STATUS_Active_count_norm,cc_balance_NAME_CONTRACT_STATUS_Approved_count,cc_balance_NAME_CONTRACT_STATUS_Approved_count_norm,cc_balance_NAME_CONTRACT_STATUS_Completed_count,cc_balance_NAME_CONTRACT_STATUS_Completed_count_norm,cc_balance_NAME_CONTRACT_STATUS_Demand_count,cc_balance_NAME_CONTRACT_STATUS_Demand_count_norm,cc_balance_NAME_CONTRACT_STATUS_Refused_count,cc_balance_NAME_CONTRACT_STATUS_Refused_count_norm,cc_balance_NAME_CONTRACT_STATUS_Sent proposal_count,cc_balance_NAME_CONTRACT_STATUS_Sent proposal_count_norm,cc_balance_NAME_CONTRACT_STATUS_Signed_count,cc_balance_NAME_CONTRACT_STATUS_Signed_count_norm
SK_ID_PREV,,,,,,,,,,,,,,
1000018,5,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1000030,8,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1000031,16,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1000035,5,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1000077,11,1.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [22]:
# Dataframe grouped by the loan
cc_balance_all = cc_balance_num_agg.merge(cc_balance_cat_agg, right_index = True, left_on = 'SK_ID_PREV', how = 'outer')

# Merge to include the SK_ID_CURR
cc_balance_all = cc_balance[['SK_ID_PREV', 'SK_ID_CURR']].merge(cc_balance_all, on = 'SK_ID_PREV', how = 'left')

# Aggregate the stats for each client
cc_balance_by_client = numeric_conversion(cc_balance_all.drop(columns = ['SK_ID_PREV']), 'SK_ID_CURR','cc')

In [23]:
cc_balance_by_client.head()

,SK_ID_CURR,cc_cc_balance_MONTHS_BALANCE_count_count,cc_cc_balance_MONTHS_BALANCE_count_mean,cc_cc_balance_MONTHS_BALANCE_count_max,cc_cc_balance_MONTHS_BALANCE_count_min,cc_cc_balance_MONTHS_BALANCE_count_sum,cc_cc_balance_MONTHS_BALANCE_mean_count,cc_cc_balance_MONTHS_BALANCE_mean_mean,cc_cc_balance_MONTHS_BALANCE_mean_max,cc_cc_balance_MONTHS_BALANCE_mean_min,...,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_count,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_mean,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_max,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_min,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_sum,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_norm_count,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_norm_mean,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_norm_max,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_norm_min,cc_cc_balance_NAME_CONTRACT_STATUS_Signed_count_norm_sum
0,100006,6,6.0,6,6,36,6,-3.5,-3.5,-3.5,...,6,0.0,0,0,0.0,6,0.0,0.0,0.0,0.0
1,100011,74,74.0,74,74,5476,74,-38.5,-38.5,-38.5,...,74,0.0,0,0,0.0,74,0.0,0.0,0.0,0.0
2,100013,96,96.0,96,96,9216,96,-48.5,-48.5,-48.5,...,96,0.0,0,0,0.0,96,0.0,0.0,0.0,0.0
3,100021,17,17.0,17,17,289,17,-10.0,-10.0,-10.0,...,17,0.0,0,0,0.0,17,0.0,0.0,0.0,0.0
4,100023,8,8.0,8,8,64,8,-7.5,-7.5,-7.5,...,8,0.0,0,0,0.0,8,0.0,0.0,0.0,0.0


In [24]:
del cc_balance_all, cc_balance_cat_agg, cc_balance_num_agg, cc_balance     # memory release
gc.collect()

119

In [25]:
# Reading the Previous Applications file from csv
prev_app = pd.read_csv('previous_application.csv', index_col=None, engine='python')
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
prev_app_num_agg = numeric_conversion(prev_app, 'SK_ID_PREV', 'prev_app')
prev_app_num_agg.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_PREV,prev_app_AMT_ANNUITY_count,prev_app_AMT_ANNUITY_mean,prev_app_AMT_ANNUITY_max,prev_app_AMT_ANNUITY_min,prev_app_AMT_ANNUITY_sum,prev_app_AMT_APPLICATION_count,prev_app_AMT_APPLICATION_mean,prev_app_AMT_APPLICATION_max,prev_app_AMT_APPLICATION_min,...,prev_app_DAYS_TERMINATION_count,prev_app_DAYS_TERMINATION_mean,prev_app_DAYS_TERMINATION_max,prev_app_DAYS_TERMINATION_min,prev_app_DAYS_TERMINATION_sum,prev_app_NFLAG_INSURED_ON_APPROVAL_count,prev_app_NFLAG_INSURED_ON_APPROVAL_mean,prev_app_NFLAG_INSURED_ON_APPROVAL_max,prev_app_NFLAG_INSURED_ON_APPROVAL_min,prev_app_NFLAG_INSURED_ON_APPROVAL_sum
0,1000001,1,6404.310,6404.310,6404.310,6404.310,1,58905.000,58905.000,58905.000,...,1,-233.0,-233.0,-233.0,-233.0,1,0.0,0.0,0.0,0.0
1,1000002,1,6264.000,6264.000,6264.000,6264.000,1,39145.500,39145.500,39145.500,...,1,-1501.0,-1501.0,-1501.0,-1501.0,1,0.0,0.0,0.0,0.0
2,1000003,1,4951.350,4951.350,4951.350,4951.350,1,47056.275,47056.275,47056.275,...,1,365243.0,365243.0,365243.0,365243.0,1,1.0,1.0,1.0,1.0
3,1000004,1,3391.110,3391.110,3391.110,3391.110,1,35144.370,35144.370,35144.370,...,1,-672.0,-672.0,-672.0,-672.0,1,0.0,0.0,0.0,0.0
4,1000005,1,14713.605,14713.605,14713.605,14713.605,1,123486.075,123486.075,123486.075,...,1,-1415.0,-1415.0,-1415.0,-1415.0,1,0.0,0.0,0.0,0.0


In [27]:
prev_app_cat_agg = categorical_conversion(prev_app, 'SK_ID_PREV', 'prev_app')
prev_app_cat_agg.head()

,prev_app_NAME_CONTRACT_TYPE_Cash loans_count,prev_app_NAME_CONTRACT_TYPE_Cash loans_count_norm,prev_app_NAME_CONTRACT_TYPE_Consumer loans_count,prev_app_NAME_CONTRACT_TYPE_Consumer loans_count_norm,prev_app_NAME_CONTRACT_TYPE_Revolving loans_count,prev_app_NAME_CONTRACT_TYPE_Revolving loans_count_norm,prev_app_NAME_CONTRACT_TYPE_XNA_count,prev_app_NAME_CONTRACT_TYPE_XNA_count_norm,prev_app_WEEKDAY_APPR_PROCESS_START_FRIDAY_count,prev_app_WEEKDAY_APPR_PROCESS_START_FRIDAY_count_norm,...,prev_app_PRODUCT_COMBINATION_POS industry without interest_count,prev_app_PRODUCT_COMBINATION_POS industry without interest_count_norm,prev_app_PRODUCT_COMBINATION_POS mobile with interest_count,prev_app_PRODUCT_COMBINATION_POS mobile with interest_count_norm,prev_app_PRODUCT_COMBINATION_POS mobile without interest_count,prev_app_PRODUCT_COMBINATION_POS mobile without interest_count_norm,prev_app_PRODUCT_COMBINATION_POS other with interest_count,prev_app_PRODUCT_COMBINATION_POS other with interest_count_norm,prev_app_PRODUCT_COMBINATION_POS others without interest_count,prev_app_PRODUCT_COMBINATION_POS others without interest_count_norm
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,
1000001,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000002,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000003,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000004,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000005,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# Dataframe grouped by the loan
prev_app_all = prev_app_num_agg.merge(prev_app_cat_agg, right_index = True, left_on = 'SK_ID_PREV', how = 'outer')

# Merge to include the SK_ID_CURR
prev_app_all = prev_app[['SK_ID_PREV', 'SK_ID_CURR']].merge(prev_app_all, on = 'SK_ID_PREV', how = 'left')

# Aggregate the stats for each client
prev_app_by_client = numeric_conversion(prev_app_all.drop(columns = ['SK_ID_PREV']), 'SK_ID_CURR','prev')
prev_app_by_client.head()

,SK_ID_CURR,prev_prev_app_AMT_ANNUITY_count_count,prev_prev_app_AMT_ANNUITY_count_mean,prev_prev_app_AMT_ANNUITY_count_max,prev_prev_app_AMT_ANNUITY_count_min,prev_prev_app_AMT_ANNUITY_count_sum,prev_prev_app_AMT_ANNUITY_mean_count,prev_prev_app_AMT_ANNUITY_mean_mean,prev_prev_app_AMT_ANNUITY_mean_max,prev_prev_app_AMT_ANNUITY_mean_min,...,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_count,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_mean,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_max,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_min,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_sum,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_norm_count,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_norm_mean,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_norm_max,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_norm_min,prev_prev_app_PRODUCT_COMBINATION_POS others without interest_count_norm_sum
0,100001,1,1.0,1,1,1,1,3951.000,3951.000,3951.000,...,1,0.0,0,0,0,1,0.0,0,0,0
1,100002,1,1.0,1,1,1,1,9251.775,9251.775,9251.775,...,1,0.0,0,0,0,1,0.0,0,0,0
2,100003,3,1.0,1,1,3,3,56553.990,98356.995,6737.310,...,3,0.0,0,0,0,3,0.0,0,0,0
3,100004,1,1.0,1,1,1,1,5357.250,5357.250,5357.250,...,1,0.0,0,0,0,1,0.0,0,0,0
4,100005,2,0.5,1,0,1,1,4813.200,4813.200,4813.200,...,2,0.0,0,0,0,2,0.0,0,0,0


In [29]:
del prev_app_all, prev_app_cat_agg, prev_app_num_agg, prev_app     # memory release
gc.collect()

91

In [30]:
# Reading the Previous Applications file from csv
pos_cash_balance = pd.read_csv('POS_CASH_BALANCE.csv', index_col=None, engine='python')
pos_cash_balance.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [31]:
pos_cash_balance_num_agg = numeric_conversion(pos_cash_balance, 'SK_ID_PREV', 'prev_app')
pos_cash_balance_num_agg.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_PREV,prev_app_MONTHS_BALANCE_count,prev_app_MONTHS_BALANCE_mean,prev_app_MONTHS_BALANCE_max,prev_app_MONTHS_BALANCE_min,prev_app_MONTHS_BALANCE_sum,prev_app_CNT_INSTALMENT_count,prev_app_CNT_INSTALMENT_mean,prev_app_CNT_INSTALMENT_max,prev_app_CNT_INSTALMENT_min,...,prev_app_SK_DPD_count,prev_app_SK_DPD_mean,prev_app_SK_DPD_max,prev_app_SK_DPD_min,prev_app_SK_DPD_sum,prev_app_SK_DPD_DEF_count,prev_app_SK_DPD_DEF_mean,prev_app_SK_DPD_DEF_max,prev_app_SK_DPD_DEF_min,prev_app_SK_DPD_DEF_sum
0,1000001,3,-9.0,-8,-10,-27,3,8.666667,12.0,2.0,...,3,0.0,0,0,0,3,0.0,0,0,0
1,1000002,5,-52.0,-50,-54,-260,5,5.200000,6.0,4.0,...,5,0.0,0,0,0,5,0.0,0,0,0
2,1000003,4,-2.5,-1,-4,-10,4,12.000000,12.0,12.0,...,4,0.0,0,0,0,4,0.0,0,0,0
3,1000004,8,-25.5,-22,-29,-204,8,9.625000,10.0,7.0,...,8,0.0,0,0,0,8,0.0,0,0,0
4,1000005,11,-51.0,-46,-56,-561,11,10.000000,10.0,10.0,...,11,0.0,0,0,0,11,0.0,0,0,0


In [32]:
pos_cash_balance_cat_agg = categorical_conversion(pos_cash_balance, 'SK_ID_PREV', 'prev_app')
pos_cash_balance_cat_agg.head()

,prev_app_NAME_CONTRACT_STATUS_Active_count,prev_app_NAME_CONTRACT_STATUS_Active_count_norm,prev_app_NAME_CONTRACT_STATUS_Amortized debt_count,prev_app_NAME_CONTRACT_STATUS_Amortized debt_count_norm,prev_app_NAME_CONTRACT_STATUS_Approved_count,prev_app_NAME_CONTRACT_STATUS_Approved_count_norm,prev_app_NAME_CONTRACT_STATUS_Canceled_count,prev_app_NAME_CONTRACT_STATUS_Canceled_count_norm,prev_app_NAME_CONTRACT_STATUS_Completed_count,prev_app_NAME_CONTRACT_STATUS_Completed_count_norm,prev_app_NAME_CONTRACT_STATUS_Demand_count,prev_app_NAME_CONTRACT_STATUS_Demand_count_norm,prev_app_NAME_CONTRACT_STATUS_Returned to the store_count,prev_app_NAME_CONTRACT_STATUS_Returned to the store_count_norm,prev_app_NAME_CONTRACT_STATUS_Signed_count,prev_app_NAME_CONTRACT_STATUS_Signed_count_norm,prev_app_NAME_CONTRACT_STATUS_XNA_count,prev_app_NAME_CONTRACT_STATUS_XNA_count_norm
SK_ID_PREV,,,,,,,,,,,,,,,,,,
1000001,2,0.666667,0,0.0,0,0.0,0,0.0,1,0.333333,0,0.0,0,0.0,0,0.0,0,0.0
1000002,4,0.800000,0,0.0,0,0.0,0,0.0,1,0.200000,0,0.0,0,0.0,0,0.0,0,0.0
1000003,4,1.000000,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0
1000004,7,0.875000,0,0.0,0,0.0,0,0.0,1,0.125000,0,0.0,0,0.0,0,0.0,0,0.0
1000005,10,0.909091,0,0.0,0,0.0,0,0.0,1,0.090909,0,0.0,0,0.0,0,0.0,0,0.0


In [33]:
# Dataframe grouped by the loan
pos_cash_balance_all = pos_cash_balance_num_agg.merge(pos_cash_balance_cat_agg, right_index = True, left_on = 'SK_ID_PREV', how = 'outer')

# Merge to include the SK_ID_CURR
pos_cash_balance_all = pos_cash_balance[['SK_ID_PREV', 'SK_ID_CURR']].merge(pos_cash_balance_all, on = 'SK_ID_PREV', how = 'left')

# Aggregate the stats for each client
pos_cash_balance_by_client = numeric_conversion(pos_cash_balance_all.drop(columns = ['SK_ID_PREV']), 'SK_ID_CURR','prev')
pos_cash_balance_by_client.head()

,SK_ID_CURR,prev_prev_app_MONTHS_BALANCE_count_count,prev_prev_app_MONTHS_BALANCE_count_mean,prev_prev_app_MONTHS_BALANCE_count_max,prev_prev_app_MONTHS_BALANCE_count_min,prev_prev_app_MONTHS_BALANCE_count_sum,prev_prev_app_MONTHS_BALANCE_mean_count,prev_prev_app_MONTHS_BALANCE_mean_mean,prev_prev_app_MONTHS_BALANCE_mean_max,prev_prev_app_MONTHS_BALANCE_mean_min,...,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_count,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_mean,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_max,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_min,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_sum,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_norm_count,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_norm_mean,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_norm_max,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_norm_min,prev_prev_app_NAME_CONTRACT_STATUS_XNA_count_norm_sum
0,100001,9,4.555556,5,4,41,9,-72.555556,-55.0,-94.5,...,9,0.0,0,0,0,9,0.0,0.0,0.0,0.0
1,100002,19,19.000000,19,19,361,19,-10.000000,-10.0,-10.0,...,19,0.0,0,0,0,19,0.0,0.0,0.0,0.0
2,100003,28,9.714286,12,8,272,28,-43.785714,-21.5,-71.5,...,28,0.0,0,0,0,28,0.0,0.0,0.0,0.0
3,100004,4,4.000000,4,4,16,4,-25.500000,-25.5,-25.5,...,4,0.0,0,0,0,4,0.0,0.0,0.0,0.0
4,100005,11,11.000000,11,11,121,11,-20.000000,-20.0,-20.0,...,11,0.0,0,0,0,11,0.0,0.0,0.0,0.0


In [34]:
del pos_cash_balance_all, pos_cash_balance_cat_agg, pos_cash_balance_num_agg, pos_cash_balance     # memory release
gc.collect()

98

In [35]:
# Reading the Installments Payments file from csv
installments_payments = pd.read_csv('installments_payments.csv', index_col=None, engine='python')
installments_payments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [36]:
# New Variable for a flag of a late payment relative to when it was supposed to have been paid
installments_payments['late'] = installments_payments['DAYS_ENTRY_PAYMENT'] > installments_payments['DAYS_INSTALMENT']
installments_payments['late'] = installments_payments['late'].astype(int)

In [37]:
installments_payments_num_agg = numeric_conversion(installments_payments, 'SK_ID_PREV', 'prev_app')
installments_payments_num_agg.head()

,SK_ID_PREV,prev_app_NUM_INSTALMENT_VERSION_count,prev_app_NUM_INSTALMENT_VERSION_mean,prev_app_NUM_INSTALMENT_VERSION_max,prev_app_NUM_INSTALMENT_VERSION_min,prev_app_NUM_INSTALMENT_VERSION_sum,prev_app_NUM_INSTALMENT_NUMBER_count,prev_app_NUM_INSTALMENT_NUMBER_mean,prev_app_NUM_INSTALMENT_NUMBER_max,prev_app_NUM_INSTALMENT_NUMBER_min,...,prev_app_AMT_PAYMENT_count,prev_app_AMT_PAYMENT_mean,prev_app_AMT_PAYMENT_max,prev_app_AMT_PAYMENT_min,prev_app_AMT_PAYMENT_sum,prev_app_late_count,prev_app_late_mean,prev_app_late_max,prev_app_late_min,prev_app_late_sum
0,1000001,2,1.500000,2.0,1.0,3.0,2,1.500000,2,1,...,2,34221.712500,62039.115,6404.31,68443.425,2,0.000000,0,0,0
1,1000002,4,1.250000,2.0,1.0,5.0,4,2.500000,4,1,...,4,9308.891250,18443.565,6264.00,37235.565,4,0.000000,0,0,0
2,1000003,3,1.000000,1.0,1.0,3.0,3,2.000000,3,1,...,3,4951.350000,4951.350,4951.35,14854.050,3,0.000000,0,0,0
3,1000004,7,1.142857,2.0,1.0,8.0,7,4.000000,7,1,...,7,4789.022143,13176.495,3391.11,33523.155,7,0.000000,0,0,0
4,1000005,11,1.000000,1.0,1.0,11.0,11,5.818182,10,1,...,11,13365.609545,14713.605,2.79,147021.705,11,0.181818,1,0,2


In [38]:
# Merge to include the SK_ID_CURR
installments_payments_all = installments_payments[['SK_ID_PREV', 'SK_ID_CURR']].merge(installments_payments_num_agg, on = 'SK_ID_PREV', how = 'left')

# Aggregate the stats for each client
installments_payments_by_client = numeric_conversion(installments_payments_all.drop(columns = ['SK_ID_PREV']), 'SK_ID_CURR','prev')
installments_payments_by_client.head()

,SK_ID_CURR,prev_prev_app_NUM_INSTALMENT_VERSION_count_count,prev_prev_app_NUM_INSTALMENT_VERSION_count_mean,prev_prev_app_NUM_INSTALMENT_VERSION_count_max,prev_prev_app_NUM_INSTALMENT_VERSION_count_min,prev_prev_app_NUM_INSTALMENT_VERSION_count_sum,prev_prev_app_NUM_INSTALMENT_VERSION_mean_count,prev_prev_app_NUM_INSTALMENT_VERSION_mean_mean,prev_prev_app_NUM_INSTALMENT_VERSION_mean_max,prev_prev_app_NUM_INSTALMENT_VERSION_mean_min,...,prev_prev_app_late_min_count,prev_prev_app_late_min_mean,prev_prev_app_late_min_max,prev_prev_app_late_min_min,prev_prev_app_late_min_sum,prev_prev_app_late_sum_count,prev_prev_app_late_sum_mean,prev_prev_app_late_sum_max,prev_prev_app_late_sum_min,prev_prev_app_late_sum_sum
0,100001,7,3.571429,4,3,25,7,1.142857,1.250000,1.000000,...,7,0.0,0,0,0,7,0.428571,1,0,3
1,100002,19,19.000000,19,19,361,19,1.052632,1.052632,1.052632,...,19,0.0,0,0,0,19,0.000000,0,0,0
2,100003,25,9.160000,12,6,229,25,1.040000,1.142857,1.000000,...,25,0.0,0,0,0,25,0.000000,0,0,0
3,100004,3,3.000000,3,3,9,3,1.333333,1.333333,1.333333,...,3,0.0,0,0,0,3,0.000000,0,0,0
4,100005,9,9.000000,9,9,81,9,1.111111,1.111111,1.111111,...,9,0.0,0,0,0,9,1.000000,1,1,9


In [39]:
del installments_payments_all, installments_payments_num_agg, installments_payments     # memory release
gc.collect()

127

In [40]:
# Read Train Data Set
application_train = pd.read_csv('application_train.csv', index_col=None, engine='python')

In [41]:
# Merging all data sets with the Train Data Set
application_train_merged = application_train.merge(prev_app_by_client, on=('SK_ID_CURR'), how='left')

In [42]:
application_train_merged = application_train_merged.merge(bureau_num_agg, on=('SK_ID_CURR'), how='left')

In [43]:
application_train_merged = application_train_merged.merge(bureau_cat_agg, on=('SK_ID_CURR'), how='left')

In [44]:
application_train_merged = application_train_merged.merge(bureau_balance_by_client, on=('SK_ID_CURR'), how='left')

In [45]:
application_train_merged = application_train_merged.merge(cc_balance_by_client, on=('SK_ID_CURR'), how='left')

In [46]:
application_train_merged = application_train_merged.merge(pos_cash_balance_by_client, on=('SK_ID_CURR'), how='left')

In [47]:
application_train_merged = application_train_merged.merge(installments_payments_by_client, on=('SK_ID_CURR'), how='left')

In [48]:
del bureau_num_agg, bureau_cat_agg, bureau_balance_by_client, cc_balance_by_client, prev_app_by_client, \
pos_cash_balance_by_client, installments_payments_by_client  # memory release
gc.collect()

7

### Now looking at missing data to decide which columns to drop and which to fill in data for

In [51]:
# Finding the number of missing variables by column 

def Missing_variables(df):
    nan_values = []
    for i in df:
        count_nan = len(df[i]) - df[i].count()
        nan_values.append(count_nan)
    percentage = [x / len(df)*100 for x in nan_values]
    nan_df = list(df.columns.values)
    percentage_list = pd.DataFrame(
    {'columns': nan_df,
     'number_Nan': nan_values,
     'percentage': percentage
    })
    
    Nan_Ascending = percentage_list.loc[percentage_list['percentage'] != 0].sort_values('percentage', ascending=False)
    
    return Nan_Ascending

In [52]:
# Before removing variables with high Nan values, making exception to EXT_SOURCE_1
# This is after seeing its high feature importance to the model
application_train_merged['EXT_SOURCE_1'].fillna((application_train_merged['EXT_SOURCE_1'].mean()), inplace=True)

In [53]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

There are 3142 columns with missing variables out of 3198 columns in the dataframe


,columns,number_Nan,percentage
364,prev_prev_app_RATE_INTEREST_PRIVILEGED_min_max,302902,98.501192
329,prev_prev_app_RATE_INTEREST_PRIMARY_mean_max,302902,98.501192
360,prev_prev_app_RATE_INTEREST_PRIVILEGED_max_min,302902,98.501192
359,prev_prev_app_RATE_INTEREST_PRIVILEGED_max_max,302902,98.501192
358,prev_prev_app_RATE_INTEREST_PRIVILEGED_max_mean,302902,98.501192
355,prev_prev_app_RATE_INTEREST_PRIVILEGED_mean_min,302902,98.501192
354,prev_prev_app_RATE_INTEREST_PRIVILEGED_mean_max,302902,98.501192
365,prev_prev_app_RATE_INTEREST_PRIVILEGED_min_min,302902,98.501192
353,prev_prev_app_RATE_INTEREST_PRIVILEGED_mean_mean,302902,98.501192
340,prev_prev_app_RATE_INTEREST_PRIMARY_min_min,302902,98.501192


In [54]:
# Creating list of column names for columns with over 50% of missing variables
# Removing these columns from the application_train_merged data set

Mising_var_high = Missing_var[Missing_var['percentage']>35]
high_Nan_column_names = Mising_var_high['columns'].tolist()

application_train_merged = application_train_merged.drop(high_Nan_column_names, axis=1)

In [55]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(20)

There are 2437 columns with missing variables out of 2493 columns in the dataframe


,columns,number_Nan,percentage
27,OCCUPATION_TYPE,96391,31.345545
1979,bureau_DAYS_ENDDATE_FACT_min,77156,25.090485
1977,bureau_DAYS_ENDDATE_FACT_mean,77156,25.090485
1978,bureau_DAYS_ENDDATE_FACT_max,77156,25.090485
2000,bureau_AMT_CREDIT_SUM_LIMIT_max,65069,21.159893
1999,bureau_AMT_CREDIT_SUM_LIMIT_mean,65069,21.159893
2001,bureau_AMT_CREDIT_SUM_LIMIT_min,65069,21.159893
42,EXT_SOURCE_3,60965,19.825307
1994,bureau_AMT_CREDIT_SUM_DEBT_mean,51380,16.708345
1995,bureau_AMT_CREDIT_SUM_DEBT_max,51380,16.708345


In [56]:
spike_cols = [col for col in application_train_merged.columns if 'Closed' in col]
print(spike_cols)

['bureau_CREDIT_ACTIVE_Closed_count', 'bureau_CREDIT_ACTIVE_Closed_count_norm']


In [57]:
# The majority of missing values are 5.350703 percentage, and those appear on those that did not merge 
# with the applicaiton_train data.
# Since there is much incomplete data - those 5% of the data will be removed for the model data-sets
# Using "late" variable for the removal. 

application_train_merged = application_train_merged[np.isfinite(application_train_merged['prev_prev_app_late_count_count'])]

In [58]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(20)

There are 2278 columns with missing variables out of 2493 columns in the dataframe


,columns,number_Nan,percentage
27,OCCUPATION_TYPE,91540,31.387690
1977,bureau_DAYS_ENDDATE_FACT_mean,72615,24.898592
1978,bureau_DAYS_ENDDATE_FACT_max,72615,24.898592
1979,bureau_DAYS_ENDDATE_FACT_min,72615,24.898592
2001,bureau_AMT_CREDIT_SUM_LIMIT_min,61131,20.960901
2000,bureau_AMT_CREDIT_SUM_LIMIT_max,61131,20.960901
1999,bureau_AMT_CREDIT_SUM_LIMIT_mean,61131,20.960901
42,EXT_SOURCE_3,57315,19.652452
1995,bureau_AMT_CREDIT_SUM_DEBT_max,48388,16.591518
1994,bureau_AMT_CREDIT_SUM_DEBT_mean,48388,16.591518


In [59]:
# The next majority of missing values are 14.277730 percentage, and those appear on those that did not merge 
# with the applicaiton_train data from the Bureau data.
# Since there is much incomplete data - those 14% of the data will be removed for the model data-sets
# Using "Closed" variable for the removal. 

application_train_merged = application_train_merged[np.isfinite(application_train_merged['bureau_DAYS_CREDIT_min'])]

In [60]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

There are 2145 columns with missing variables out of 2493 columns in the dataframe


,columns,number_Nan,percentage
27,OCCUPATION_TYPE,78132,31.252425
1977,bureau_DAYS_ENDDATE_FACT_mean,30975,12.389851
1978,bureau_DAYS_ENDDATE_FACT_max,30975,12.389851
1979,bureau_DAYS_ENDDATE_FACT_min,30975,12.389851
2001,bureau_AMT_CREDIT_SUM_LIMIT_min,19491,7.796306
2000,bureau_AMT_CREDIT_SUM_LIMIT_max,19491,7.796306
1999,bureau_AMT_CREDIT_SUM_LIMIT_mean,19491,7.796306
42,EXT_SOURCE_3,16879,6.751519
262,prev_prev_app_RATE_DOWN_PAYMENT_max_min,15522,6.208725
265,prev_prev_app_RATE_DOWN_PAYMENT_min_mean,15522,6.208725


In [61]:
# The next majority of missing values are 0.855590 percentage, and those appear on those that did not merge 
# with the applicaiton_train data from the Bureau data.
# Since there is much incomplete data - those 0.8% of the data will be removed for the model data-sets
# Using "Closed" variable for the removal. 

application_train_merged = application_train_merged[np.isfinite(application_train_merged['prev_prev_app_SK_DPD_min_count'])]

In [62]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

There are 1948 columns with missing variables out of 2493 columns in the dataframe


,columns,number_Nan,percentage
27,OCCUPATION_TYPE,77505,31.268533
1979,bureau_DAYS_ENDDATE_FACT_min,30548,12.324252
1978,bureau_DAYS_ENDDATE_FACT_max,30548,12.324252
1977,bureau_DAYS_ENDDATE_FACT_mean,30548,12.324252
1999,bureau_AMT_CREDIT_SUM_LIMIT_mean,19192,7.742800
2000,bureau_AMT_CREDIT_SUM_LIMIT_max,19192,7.742800
2001,bureau_AMT_CREDIT_SUM_LIMIT_min,19192,7.742800
42,EXT_SOURCE_3,16706,6.739850
165,prev_prev_app_AMT_DOWN_PAYMENT_min_mean,13797,5.566247
156,prev_prev_app_AMT_DOWN_PAYMENT_mean_max,13797,5.566247


In [63]:
# many records with 0.378829 percent of missing records. removing.

application_train_merged = application_train_merged[np.isfinite(application_train_merged['prev_prev_app_AMT_ANNUITY_mean_max'])]

In [64]:
Missing_var['percentage'].value_counts()

0.337678     1788
0.517612       54
0.002017       18
0.002421       18
5.566247       18
0.378426        9
0.378829        9
0.523260        9
0.270304        4
7.742800        3
2.691341        3
0.823822        3
0.000403        3
12.324252       3
31.268533       1
0.089160        1
6.739850        1
0.185178        1
0.004438        1
0.269094        1
Name: percentage, dtype: int64

In [65]:
# 54 records with 0.139716 percent of missing records. removing.

application_train_merged = application_train_merged[np.isfinite(application_train_merged['prev_prev_app_DAYS_FIRST_DUE_min_min'])]

In [66]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(10)

There are 88 columns with missing variables out of 2493 columns in the dataframe


,columns,number_Nan,percentage
27,OCCUPATION_TYPE,77079,31.258592
1978,bureau_DAYS_ENDDATE_FACT_max,30382,12.321106
1979,bureau_DAYS_ENDDATE_FACT_min,30382,12.321106
1977,bureau_DAYS_ENDDATE_FACT_mean,30382,12.321106
1999,bureau_AMT_CREDIT_SUM_LIMIT_mean,19090,7.741752
2000,bureau_AMT_CREDIT_SUM_LIMIT_max,19090,7.741752
2001,bureau_AMT_CREDIT_SUM_LIMIT_min,19090,7.741752
42,EXT_SOURCE_3,16612,6.736825
262,prev_prev_app_RATE_DOWN_PAYMENT_max_min,12841,5.207535
267,prev_prev_app_RATE_DOWN_PAYMENT_min_min,12841,5.207535


In [67]:
# Removing rest of records with <0.1 percent of missing records. removing.

application_train_merged = application_train_merged[np.isfinite(application_train_merged['bureau_AMT_CREDIT_SUM_max'])]
application_train_merged = application_train_merged[np.isfinite(application_train_merged['prev_prev_app_CNT_INSTALMENT_FUTURE_max_max'])]
application_train_merged = application_train_merged[np.isfinite(application_train_merged['AMT_ANNUITY'])]
application_train_merged = application_train_merged[np.isfinite(application_train_merged['AMT_GOODS_PRICE'])]
application_train_merged = application_train_merged[np.isfinite(application_train_merged['prev_prev_app_AMT_GOODS_PRICE_max_min'])]
application_train_merged = application_train_merged[np.isfinite(application_train_merged['prev_prev_app_DAYS_ENTRY_PAYMENT_min_max'])]

In [68]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')
Missing_var.head(50)

There are 38 columns with missing variables out of 2493 columns in the dataframe


,columns,number_Nan,percentage
27,OCCUPATION_TYPE,76888,31.263545
1979,bureau_DAYS_ENDDATE_FACT_min,30282,12.313010
1978,bureau_DAYS_ENDDATE_FACT_max,30282,12.313010
1977,bureau_DAYS_ENDDATE_FACT_mean,30282,12.313010
2001,bureau_AMT_CREDIT_SUM_LIMIT_min,19039,7.741476
2000,bureau_AMT_CREDIT_SUM_LIMIT_max,19039,7.741476
1999,bureau_AMT_CREDIT_SUM_LIMIT_mean,19039,7.741476
42,EXT_SOURCE_3,16555,6.731453
255,prev_prev_app_RATE_DOWN_PAYMENT_mean_mean,12421,5.050521
267,prev_prev_app_RATE_DOWN_PAYMENT_min_min,12421,5.050521


In [69]:
# Filling NaN values in OCCUPATION_TYPE as "Unemployed"
application_train_merged['OCCUPATION_TYPE'].fillna('Unemployed', inplace=True)

# Filling NaN values in EXT_Source_3 and _2 with the mean of each.
application_train_merged['EXT_SOURCE_3'].fillna((application_train_merged['EXT_SOURCE_3'].mean()), inplace=True)
application_train_merged['EXT_SOURCE_2'].fillna((application_train_merged['EXT_SOURCE_2'].mean()), inplace=True)

# Filling NaN values in NAME_TYPE_SUITE as "Unaccompanied"
application_train_merged['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace=True)

# Filling NaN values in XXX_CNT_SOCIAL_CIRCLE as "0.0". Making assumption that not listed is not observed.
# These are defined as "How many observation of client's social surroundings defaulted on 30 DPD (days past due)""
application_train_merged['OBS_30_CNT_SOCIAL_CIRCLE'].fillna('0.0', inplace=True)
application_train_merged['DEF_30_CNT_SOCIAL_CIRCLE'].fillna('0.0', inplace=True)
application_train_merged['OBS_60_CNT_SOCIAL_CIRCLE'].fillna('0.0', inplace=True)
application_train_merged['DEF_60_CNT_SOCIAL_CIRCLE'].fillna('0.0', inplace=True)

In [70]:
# Filling rest of the values with mean of the column
Missing_var = Missing_variables(application_train_merged)
columns = Missing_var['columns'].tolist()
for i in columns:
    application_train_merged[i].fillna((application_train_merged[i].mean()), inplace=True)

In [71]:
Missing_var = Missing_variables(application_train_merged)
print('There are', len(Missing_var), 'columns with missing variables out of', \
      len(application_train_merged.columns), 'columns in the dataframe')

There are 0 columns with missing variables out of 2493 columns in the dataframe


### Moving to look at outlier values... Continued on the EDA notebook

In [72]:
application_train_merged.to_csv('application_train_merged_FINAL.csv',index=False)